In [ ]:
# INSTALLS THAT NEED TO BE RUN ON CONDA
# !pip install loguru

In [ ]:
import json
import boto3
import base64
import re
import os
from PIL import Image
from pathlib import Path
import glob
import time
from importlib import reload
import pandas as pd
from IPython.display import display
from io import BytesIO
from botocore.config import Config
import logging

logging.basicConfig(level=logging.INFO, force=True)  # Resets handlers

%load_ext autoreload
%autoreload 2

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# os.chdir('..')
print("CWD:", os.getcwd())
bedrock_runtime = boto3.client("bedrock-runtime")
s3 = boto3.client("s3")

In [ ]:
try:
    os.chdir("../../lib/src")
    import image_captioning_assistant.generate.prompts as p
    import image_captioning_assistant.data.data_classes as dc
    
    # current
    import image_captioning_assistant.generate.bias_analysis.find_biases_in_short_work as gbsw
    import image_captioning_assistant.generate.bias_analysis.find_biases_in_long_work as gblw
    import image_captioning_assistant.generate.utils as gen_utils

    # import image_captioning_assistant.data.data_classes as dc
    import image_captioning_assistant.aws.s3 as s3_util
finally:
    os.chdir("../../projects/research")

In [ ]:
# Helper functions for notebook
def show_base64_image(encoded_str):
    # Add padding if missing
    missing_padding = len(encoded_str) % 4
    if missing_padding:
        encoded_str += "=" * (4 - missing_padding)

    # Decode and display
    image_data = base64.b64decode(encoded_str)
    image = Image.open(BytesIO(image_data))
    display(image)


def download_s3_directory(bucket, s3_prefix, local_dir):
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    for page in paginator.paginate(Bucket=bucket, Prefix=s3_prefix):
        for obj in page.get("Contents", []):
            # Skip directory markers
            if obj["Key"].endswith("/"):
                continue

            # Build local path
            relative_path = obj["Key"].replace(s3_prefix, "", 1)
            local_path = local_dir / relative_path

            # Create parent directories and download
            local_path.parent.mkdir(parents=True, exist_ok=True)
            s3.download_file(bucket, obj["Key"], str(local_path))


def display_bias(bias_list, attribute):
    # Convert to DataFrame with multi-index
    multi_index_data = []
    for i, bias_dict in enumerate(bias_list):
        for key, value in bias_dict.items():
            multi_index_data.append(((i + 1, key), value))

    # Create DataFrame
    multi_index = pd.MultiIndex.from_tuples([item[0] for item in multi_index_data], names=["Bias ID", "Bias Item"])
    df = pd.DataFrame(
        {f"Output from AI Model for {attribute}": [item[1] for item in multi_index_data]},
        index=multi_index,
    )
    display(df)


def print_output(output):
    if "description" in output:
        s = pd.Series(output)
        display(pd.DataFrame({"Metadata Item": s.index, "Output from AI Model": s.values}))
    else:
        display_bias(output["metadata_biases"]["biases"], "Metadata")
        for i, bias_list in enumerate(output["page_biases"]):
            display_bias(bias_list["biases"], f"Page {i+1}")


def display_work_id_images(work_id):
    """
    Display images for a work ID from S3, without saving to disk.
    """
    # Get SHA1s for the work_id
    shas = ground_truth_df[ground_truth_df["work_id"] == work_id]["page_sha1"]
    
    # Create S3 URIs
    image_s3_uris = [f"s3://{bucket_name}/ground_truth_images/{sha}" for sha in shas]
    
    # Load all images with no resizing
    image_bytes_list = gen_utils.load_and_resize_images(
        image_s3_uris=image_s3_uris,
        s3_kwargs={},
        resize_kwargs={}  # No resizing as requested
    )
    
    # Display the images
    from io import BytesIO
    
    for img_bytes in image_bytes_list:
        if img_bytes is not None:
            img = Image.open(BytesIO(img_bytes))
            display(img)
        else:
            print("Failed to load image")


def gen_bias_for_wid(work_id, page_title, model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0"):
    image_path = "ground_truth/images"
    shas = ground_truth_df[ground_truth_df["work_id"] == work_id][["page_sha1", "page_title"]]
    image_s3_uris = []
    if page_title.lower() == "front" and len(shas) > 1:
        front_sha = shas[shas["page_title"].str.lower() == "front"]["page_sha1"].values[0]
        image_s3_uris.append(f"s3://gaiic-emory-dev/ground_truth_images/{front_sha}")
        back_sha = shas[shas["page_title"].str.lower() == "back"]["page_sha1"].values[0]
        image_s3_uris.append(f"s3://gaiic-emory-dev/ground_truth_images/{back_sha}")
    else:
        front_sha = shas[shas["page_title"] == page_title]["page_sha1"].values[0]
        image_s3_uris.append(f"s3://gaiic-emory-dev/ground_truth_images/{front_sha}")
        back_sha = None

    s3_kwargs = {
        "config": Config(
            s3={"addressing_style": "virtual"},
            signature_version="s3v4",
        ),
        "region_name": "us-east-1",
    }

    llm_kwargs = {
        # "model": "anthropic.claude-3-5-sonnet-20240620-v1:0",
        "model_id": model_id,
    }

    return gbsw.find_biases_in_short_work(
        image_s3_uris,
        s3_kwargs,
        llm_kwargs,
        {},
        # work_context: str | None = None,
        # original_metadata: str | None = None,
    )


def gen_bias_for_wid_long(work_id, model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0"):
    image_path = "ground_truth/images"
    sha_df = ground_truth_df[ground_truth_df["work_id"] == work_id][["page_sha1", "page_title"]]
    shas = list(sha_df["page_sha1"].values)
    image_page_names = list(sha_df["page_title"].values)
    image_s3_uris = [f"s3://gaiic-emory-dev/ground_truth_images/{sha}" for sha in shas]

    s3_kwargs = {
        "config": Config(
            s3={"addressing_style": "virtual"},
            signature_version="s3v4",
        ),
    }

    llm_kwargs = {
        # "model": "anthropic.claude-3-5-sonnet-20240620-v1:0",
        "model_id": model_id,
        "region_name": "us-east-1",
    }

    return gblw.find_biases_in_long_work(
        image_s3_uris,
        s3_kwargs,
        llm_kwargs,
        {},
        # work_context: str | None = None,
        # original_metadata: str | None = None,
    )

In [ ]:
# download ground truth set to local

# Configuration
bucket_name = "gaiic-emory-dev"
local_base = Path("ground_truth")
s3_client = boto3.client("s3")

# Download single CSV file
csv_path = local_base / "ground_truth.csv"
csv_path.parent.mkdir(parents=True, exist_ok=True)
s3_client.download_file(bucket_name, "ground_truth.csv", str(csv_path))
ground_truth_df = pd.read_csv("ground_truth/ground_truth.csv")

# Download entire images directory -- slow, not required
# download_s3_directory(
#     bucket=bucket_name,
#     s3_prefix='ground_truth_images/',
#     local_dir=local_base / 'images'
# )

In [ ]:
# OPTIONAL for diagnostics, dump out the WorkBiasAnalysis model schema, which is what is returned by the bias models
# formatted_schema = json.dumps(dc.WorkBiasAnalysis.model_json_schema(), indent=4, ensure_ascii=True)
# print(formatted_schema)

In [ ]:
# Visualize a WID
display_work_id_images("689d51c5f7-cor")

In [ ]:
# Generate the bias output and visualize it
model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
# results = gen_bias_for_wid("880ht76hj7-cor", "Front", model_id=model_id)  # cotton in sunny south 689d51c5f7-cor
# results = gen_bias_for_wid("989r2280h5-cor", "Front", model_id=model_id)  # Hatian mother with offspring
results = gen_bias_for_wid("689d51c5f7-cor", "Front", model_id=model_id)  # AA boy pointing at possum in tree
# results = gen_bias_for_wid('24298sf7s0-cor', "Front", model_id=model_id)  # Burning of AA man
# print(results.cot)
print_output(results.model_dump())

In [ ]:
# Generate the bias output and visualize it
model_id = "amazon.nova-pro-v1:0"
# results = gen_bias_for_wid("880ht76hj7-cor", "Front", model_id=model_id)  # cotton in sunny south 689d51c5f7-cor
# results = gen_bias_for_wid("989r2280h5-cor", "Front", model_id=model_id)  # Hatian mother with offspring
results = gen_bias_for_wid("689d51c5f7-cor", "Front", model_id=model_id)  # AA boy pointing at possum in tree
# results = gen_bias_for_wid('24298sf7s0-cor', "Front", model_id=model_id)  # Burning of AA man
# print(results.cot)
print_output(results.model_dump())

## Run for longer works

In [ ]:
display_work_id_images("26663xsjkv-cor")

In [ ]:
model_id = "amazon.nova-pro-v1:0"
# response = gen_bias_for_wid_long('648ffbg7pg-cor') # 3 pages (high, none, low)
response = gen_bias_for_wid_long("26663xsjkv-cor", model_id=model_id)  # 4 pages (medium, high, none, none)
print_output(response.model_dump())

In [ ]:
model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
# response = gen_bias_for_wid_long('648ffbg7pg-cor') # 3 pages (high, none, low)
response = gen_bias_for_wid_long("26663xsjkv-cor", model_id=model_id)  # 4 pages (medium, high, none, none)
print_output(response.model_dump())

In [ ]:
model_id = "us.meta.llama3-2-90b-instruct-v1:0"
# response = gen_bias_for_wid_long('648ffbg7pg-cor') # 3 pages (high, none, low)
response = gen_bias_for_wid_long("26663xsjkv-cor", model_id=model_id)  # 4 pages (medium, high, none, none)
print_output(response.model_dump())

# Run Evaluation -- NEEDS FIXING TO DEAL WITH DIFFERENCES BETWEEN GROUND TRUTH AND GENERATIONS

In [ ]:
try:
    os.chdir("../../lib/src")
    import image_captioning_assistant.evaluate.evaluate_bias_analysis as eba
    import image_captioning_assistant.evaluate.evaluate_structured_metadata as esm
    import image_captioning_assistant.evaluate.evaluate_freeform_description as efd
    from image_captioning_assistant.data.constants import BiasLevel, BiasType, LibraryFormat
finally:
    os.chdir("../../projects/research")

### Bias Eval

In [ ]:
llm_bias_analysis = dc.BiasAnalysisEntry(
    bias_type=BiasType.racial,
    bias_level=BiasLevel.high,
    explanation="Water fountain and a sign above it that says 'whites'",
)
human_bias_analysis = dc.BiasAnalysisEntry(
    bias_type=BiasType.racial,
    bias_level=BiasLevel.high,
    explanation="A water fountain and a sign above it tha reads 'whites'",
)

potential_bias_evaluation = eba.evaluate_potential_biases(
    llm_potential_biases=[llm_bias_analysis],
    human_potential_biases=[human_bias_analysis],
    chat_bedrock_converse_kwargs={
        "model": "anthropic.claude-3-5-sonnet-20240620-v1:0",
        "temperature": 0.0,
    },
)
potential_bias_evaluation

In [ ]:
llm_bias_analysis = dc.BiasAnalysisEntry(
    bias_type=BiasType.racial,
    bias_level=BiasLevel.high,
    explanation="Water fountain and a sign above it that says 'whites'",
)
human_bias_analysis = dc.BiasAnalysisEntry(
    bias_type=BiasType.age,
    bias_level=BiasLevel.low,
    explanation="Child Laborers are working in the fields.",
)

potential_bias_evaluation = eba.evaluate_potential_biases(
    llm_potential_biases=[llm_bias_analysis],
    human_potential_biases=[human_bias_analysis],
    chat_bedrock_converse_kwargs={
        "model": "anthropic.claude-3-5-sonnet-20240620-v1:0",
        "temperature": 0.0,
    },
)
potential_bias_evaluation